* we will replicate the code in the section 6 `PyTorch Custom Dataset` and transfrom it into modulers `.py` files

# 1. Get Data

In [1]:
import os
import requests
import zipfile
from pathlib import Path


TARGET_URL = "https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip"

data_path = Path("data/")
image_path = data_path / "pizza_steak_sushi"

file_path = data_path / TARGET_URL.split("/")[-1]

if image_path.is_dir():
    print(f"{image_path} directory already exists, skipping download...")
    
else:
    print(f"{image_path} does not exist, creating one...")
    image_path.mkdir(parents=True, exist_ok=True)
    
    with open(file_path, "wb") as f:
        response = requests.get(TARGET_URL)
        print("Downloading pizza steak sushi data...")

        f.write(response.content)

    with zipfile.ZipFile(file_path, "r") as zip_ref:
        print("Unzipping pizza steak sushi data...")
        zip_ref.extractall(image_path)
        
    os.remove(file_path) # Remove the folder after unzipping it.

data\pizza_steak_sushi directory already exists, skipping download...


In [2]:
# Setup train & testing paths
train_dir =  image_path / "train"
test_dir = image_path / "test"

train_dir, test_dir

(WindowsPath('data/pizza_steak_sushi/train'),
 WindowsPath('data/pizza_steak_sushi/test'))

# 2. Create Datasets & DataLoaders

In [3]:
from torchvision import datasets, transforms

data_transform = transforms.Compose([
    transforms.Resize(size=(64, 64)),
    transforms.ToTensor()
])

train_data = datasets.ImageFolder(
    root=train_dir,
    transform=data_transform,
    target_transform=None,
)

test_data = datasets.ImageFolder(
    root=test_dir,
    transform=data_transform,
    target_transform=None,
)

print(f"Train Data:\n {train_data}\nTesting Data:\n {test_data}")

Train Data:
 Dataset ImageFolder
    Number of datapoints: 225
    Root location: data\pizza_steak_sushi\train
    StandardTransform
Transform: Compose(
               Resize(size=(64, 64), interpolation=bilinear, max_size=None, antialias=True)
               ToTensor()
           )
Testing Data:
 Dataset ImageFolder
    Number of datapoints: 75
    Root location: data\pizza_steak_sushi\test
    StandardTransform
Transform: Compose(
               Resize(size=(64, 64), interpolation=bilinear, max_size=None, antialias=True)
               ToTensor()
           )


In [4]:
from torch.utils.data import DataLoader
import multiprocessing

num_workers = multiprocessing.cpu_count()
BATCH_SIZE = 4

train_dataloader = DataLoader(
    dataset=train_data,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=num_workers,
)

test_dataloader = DataLoader(
    dataset=test_data,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=num_workers,
)


train_dataloader, test_dataloader

(<torch.utils.data.dataloader.DataLoader at 0x1590b03de90>,
 <torch.utils.data.dataloader.DataLoader at 0x1590b03e450>)

# 2.1 Create Datasets & DataLoaders (Script Mode)

* We  will use the Jupyter magic functions to create a `.py` file for creating dataloaders.
* We can save a code cell's contents to a file using the Jupyter magic `%%writefile filename`.

In [5]:
# Creatr a folder which will contain our moduler
import os
os.makedirs("modulers")

In [23]:
%%writefile "modulers\data_setup.py"
import multiprocessing

from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# NUM_WORKERS = multiprocessing.cpu_count()

def create_dataloader(
    train_dir: str,
    test_dir: str,
    transform: transforms.Compose,
    batch_size: int,
#     NUM_WORKERS: int = NUM_WORKERS,
):
    """
    Creates DataLoader objects for training and testing datasets.

    Args:
        train_dir (str): Path to the directory containing training images.
        test_dir (str): Path to the directory containing testing images.
        transform (transforms.Compose): A torchvision transforms.Compose object containing the transformations to apply to the images.
        NUM_WORKERS (int, optional): Number of worker processes to use for data loading. Defaults to the number of available CPU cores.
        batch_size (int): Number of images to process in each batch.

    Returns:
        tuple: A tuple containing:
            - train_loader (DataLoader): DataLoader for the training dataset.
            - test_loader (DataLoader): DataLoader for the testing dataset.
            - class_names (list): List of class names found in the training directory.

    Example usage:
        train_dir = "/path/to/train"
        test_dir = "/path/to/test"
        transform = transforms.Compose([...])
        batch_size = 32
        
        train_loader, test_loader, class_names = create_dataloader(train_dir, test_dir, transform, batch_size=batch_size)
    """
    # Read the image data from the Folders
    train_data = datasets.ImageFolder(root=train_dir, transform=transform, )
    test_data = datasets.ImageFolder(root=test_dir, transform=transform)
    
    class_names = train_data.classes
    
    # Put the data into loaders
    train_loader = DataLoader(
        train_data, 
        batch_size=batch_size, 
#         num_workers=NUM_WORKERS,
        shuffle=True,
        pin_memory=True
    )
    test_loader = DataLoader(
        test_data, 
        batch_size=batch_size, 
#         num_workers=NUM_WORKERS,
        pin_memory=True
    )

    return train_loader, test_loader, class_names

Overwriting modulers\data_setup.py


In [9]:
import importlib
import modulers.data_setup

importlib.reload(modulers.data_setup)

from modulers.data_setup import create_dataloader

train_loader, test_loader, class_names = create_dataloader(
    train_dir=train_dir,
    test_dir=test_dir,
    transform=data_transform,
    batch_size=32,
)

# 3. Making a model (TinyVGG)

In [10]:
import torch

class TinyVGG(torch.nn.Module):
    def __init__(self, input_shape: int, hidden_units: int, output_shape: int) -> None:
        super().__init__()
        
        self.conv_block_1 = torch.nn.Sequential(
            torch.nn.Conv2d(in_channels=input_shape, out_channels=hidden_units, kernel_size=3, padding=0),
            torch.nn.ReLU(),
            torch.nn.Conv2d(in_channels=hidden_units, out_channels=hidden_units, kernel_size=3, padding=0),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        self.conv_block_2 = torch.nn.Sequential(
            torch.nn.Conv2d(in_channels=hidden_units, out_channels=hidden_units, kernel_size=3, padding=0),
            torch.nn.ReLU(),
            torch.nn.Conv2d(in_channels=hidden_units, out_channels=hidden_units, kernel_size=3, padding=0),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        self.classifier = torch.nn.Sequential(
            torch.nn.Flatten(),
            torch.nn.Linear(in_features=13*13*hidden_units, out_features=output_shape)
        )
        
    def forward(self, x):
        return self.classifier(self.conv_block_2(self.conv_block_1(x)))

In [11]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_0 = TinyVGG(
    input_shape=3,
    hidden_units=10,
    output_shape=len(train_data.classes),
).to(device)

# 3.1. Making a model `TinyVGG` with a script `model_builder.py`

In [12]:
%%writefile "modulers\model_builder.py"

import torch
    
class TinyVGG(torch.nn.Module):
    """
    Creates a simple feedforward neural network model.

    Args:
        input_shape (int): The number of input features for the model.
        hidden_units (int): The number of units in the hidden layer.
        output_shape (int): The number of output units, typically corresponding to the number of classes for classification.

    Example usage:
        input_shape = 784
        hidden_units = 128
        output_shape = 10

        model = TinyVGG(input_shape, hidden_units, output_shape)
    """
    def __init__(self, input_shape: int, hidden_units: int, output_shape: int) -> None:
        super().__init__()

        self.conv_block_1 = torch.nn.Sequential(
            torch.nn.Conv2d(in_channels=input_shape, out_channels=hidden_units, kernel_size=3, padding=0),
            torch.nn.ReLU(),
            torch.nn.Conv2d(in_channels=hidden_units, out_channels=hidden_units, kernel_size=3, padding=0),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.conv_block_2 = torch.nn.Sequential(
            torch.nn.Conv2d(in_channels=hidden_units, out_channels=hidden_units, kernel_size=3, padding=0),
            torch.nn.ReLU(),
            torch.nn.Conv2d(in_channels=hidden_units, out_channels=hidden_units, kernel_size=3, padding=0),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.classifier = torch.nn.Sequential(
            torch.nn.Flatten(),
            torch.nn.Linear(in_features=13*13*hidden_units, out_features=output_shape)
        )

    def forward(self, x):
        return self.classifier(self.conv_block_2(self.conv_block_1(x)))

Overwriting modulers\model_builder.py


In [13]:
import importlib
import modulers.model_builder

importlib.reload(modulers.model_builder)

from modulers.model_builder import TinyVGG

model_1 = TinyVGG(
    input_shape=3, # in my case it will be three, cause the color channel of my data is 3
    hidden_units=10,
    output_shape=len(train_data.classes),
).to(device)

model_1

TinyVGG(
  (conv_block_1): Sequential(
    (0): Conv2d(3, 10, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv_block_2): Sequential(
    (0): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=1690, out_features=3, bias=True)
  )
)

# 4. Creating `train_step()`,  `test_step()` functions and `train()` to combine them

In [14]:
from typing import Tuple

def train_step(
    model: torch.nn.Module,
    dataloader: torch.utils.data.DataLoader,
    optimizer: torch.optim.Optimizer,
    loss_fn: torch.nn.Module,
    device=device
) -> Tuple[float, float]:
    model.train()
    train_loss, train_acc = 0, 0
    
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        
        # 1. Forward Pass
        output = model(X) # outputs will be a logits
        
        # 2. Calculate the loss
        loss = loss_fn(output, y)
        train_loss += loss.item() # loss per batch
        
        #3. Optimizer zero grad
        optimizer.zero_grad()
        
        # 4. backward pass
        loss.backward()
        
        # 5. Optimizer step
        optimizer.step()
        
        # Calculate the acc
        y_pred_classes = torch.argmax(torch.softmax(output, dim=-1), dim=-1)
        train_acc += ((y_pred_classes == y).sum() / len(y)).item() # acc per batch
        
#     adjust metrics to get all loss/accuracy values per all of the data
    train_loss /= len(dataloader)
    train_acc /= len(dataloader)
    
    return train_acc, train_loss

In [15]:
def test_step(
    model: torch.nn.Module,
    dataloader: torch.utils.data.DataLoader,
    loss_fn: torch.nn.Module,
    device=device
) -> Tuple[float, float]:
    
    model.eval()
    test_loss, test_acc = 0, 0
    
    with torch.inference_mode():
        for batch, (X, y) in enumerate(dataloader):
            
            X, y = X.to(device), y.to(device)
            outputs = model(X)

            loss = loss_fn(outputs, y)
            
            test_loss += loss.item()
            
            y_pred_classes = torch.argmax(torch.softmax(outputs, dim=-1), dim=-1)
            test_acc += (y_pred_classes == y).sum().item() / len(y)
            
    test_loss /= len(dataloader)
    test_acc /= len(dataloader)
    
    return test_acc, test_loss

In [16]:
from typing import List, Dict

from tqdm.auto import tqdm

def train(
    model: torch.nn.Module,
    train_dataloader: torch.utils.data.DataLoader,
    test_dataloader: torch.utils.data.DataLoader,
    loss_fn: torch.nn.Module,
    optimizer: torch.optim.Optimizer,
    epochs: int = 5,
    device: torch.device = device
) -> Dict[str, List[float]]:
#     Result dictionary
    result = {
        "train_acc": [],
        "train_loss": [],
        "test_acc": [],
        "test_loss": [],
    }
#     loop throught train, and test step functions
    for epoch in tqdm(range(epochs)):
        
        train_acc, train_loss = train_step(model, train_dataloader, optimizer, loss_fn, device)
        
        test_acc, test_loss = test_step(model, test_dataloader, loss_fn, device)
        
#         print out the result
        print(
            f"Epoch: {epoch} | Train acc: {train_acc:.4f}, Train Loss: {train_loss:.4f} | Test acc: {test_acc:.4f}, Test Loss: {test_loss:.4f}"
        )
    
        result["train_acc"].append(train_acc)
        result["train_loss"].append(train_loss)
        result["test_acc"].append(test_acc)
        result["test_loss"].append(test_loss)
    return result


# 4.1. Turn training functions into a script `engine.py`

In [17]:
%%writefile "modulers\engine.py"

from typing import Tuple, List, Dict

from tqdm.auto import tqdm
import torch

def train_step(
    model: torch.nn.Module,
    dataloader: torch.utils.data.DataLoader,
    optimizer: torch.optim.Optimizer,
    loss_fn: torch.nn.Module,
    device: torch.device
) -> Tuple[float, float]:
    
    """
    Performs a single training step for one epoch.

    Args:
        model (torch.nn.Module): The neural network model to train.
        dataloader (torch.utils.data.DataLoader): DataLoader providing the training data in batches.
        optimizer (torch.optim.Optimizer): The optimizer used for updating the model's parameters.
        loss_fn (torch.nn.Module): The loss function used to calculate the error between predictions and targets.
        device (torch.device): The device (CPU or GPU) on which the computations are performed.

    Returns:
        Tuple[float, float]: A tuple containing:
            - train_acc (float): The average training accuracy across all batches.
            - train_loss (float): The average training loss across all batches.

    Example usage:
        train_acc, train_loss = train_step(model, train_dataloader, optimizer, loss_fn, device)
    """
    
    model.train()
    train_loss, train_acc = 0, 0
    
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        
        # 1. Forward Pass
        output = model(X) # outputs will be a logits
        
        # 2. Calculate the loss
        loss = loss_fn(output, y)
        train_loss += loss.item() # loss per batch
        
        #3. Optimizer zero grad
        optimizer.zero_grad()
        
        # 4. backward pass
        loss.backward()
        
        # 5. Optimizer step
        optimizer.step()
        
        # Calculate the acc
        y_pred_classes = torch.argmax(torch.softmax(output, dim=-1), dim=-1)
        train_acc += ((y_pred_classes == y).sum() / len(y)).item() # acc per batch
        
#     adjust metrics to get all loss/accuracy values per all of the data
    train_loss /= len(dataloader)
    train_acc /= len(dataloader)
    
    return train_acc, train_loss


def test_step(
    model: torch.nn.Module,
    dataloader: torch.utils.data.DataLoader,
    loss_fn: torch.nn.Module,
    device: torch.device
) -> Tuple[float, float]:
    """
    Evaluates the model on a validation or test dataset.

    Args:
        model (torch.nn.Module): The neural network model to evaluate.
        dataloader (torch.utils.data.DataLoader): DataLoader providing the validation or test data in batches.
        loss_fn (torch.nn.Module): The loss function used to calculate the error between predictions and targets.
        device (torch.device): The device (CPU or GPU) on which the computations are performed.

    Returns:
        Tuple[float, float]: A tuple containing:
            - test_acc (float): The average accuracy across all batches in the test dataset.
            - test_loss (float): The average loss across all batches in the test dataset.

    Example usage:
        test_acc, test_loss = test_step(model, test_dataloader, loss_fn, device)
    """
    
    model.eval()
    test_loss, test_acc = 0, 0
    
    with torch.inference_mode():
        for batch, (X, y) in enumerate(dataloader):
            
            X, y = X.to(device), y.to(device)
            outputs = model(X)

            loss = loss_fn(outputs, y)
            
            test_loss += loss.item()
            
            y_pred_classes = torch.argmax(torch.softmax(outputs, dim=-1), dim=-1)
            test_acc += (y_pred_classes == y).sum().item() / len(y)
            
    test_loss /= len(dataloader)
    test_acc /= len(dataloader)
    
    return test_acc, test_loss


def train(
    model: torch.nn.Module,
    train_dataloader: torch.utils.data.DataLoader,
    test_dataloader: torch.utils.data.DataLoader,
    loss_fn: torch.nn.Module,
    optimizer: torch.optim.Optimizer,
    epochs: int,
    device: torch.device
) -> Dict[str, List[float]]:
    
    """
    Trains and evaluates the model over a specified number of epochs.

    Args:
        model (torch.nn.Module): The neural network model to train and evaluate.
        train_dataloader (torch.utils.data.DataLoader): DataLoader providing the training data in batches.
        test_dataloader (torch.utils.data.DataLoader): DataLoader providing the test data in batches.
        loss_fn (torch.nn.Module): The loss function used to calculate the error between predictions and targets.
        optimizer (torch.optim.Optimizer): The optimizer used for updating the model's parameters.
        epochs (int, optional): The number of epochs to train the model. Defaults to 5.
        device (torch.device, optional): The device (CPU or GPU) on which the computations are performed. Defaults to the global `device`.

    Returns:
        Dict[str, List[float]]: A dictionary containing lists of accuracy and loss values for both the training and test datasets across all epochs:
            - "train_acc" (List[float]): Training accuracy for each epoch.
            - "train_loss" (List[float]): Training loss for each epoch.
            - "test_acc" (List[float]): Test accuracy for each epoch.
            - "test_loss" (List[float]): Test loss for each epoch.

    Example usage:
        results = train(model, train_dataloader, test_dataloader, loss_fn, optimizer, epochs=10, device=device)
    """
#     Result dictionary
    result = {
        "train_acc": [],
        "train_loss": [],
        "test_acc": [],
        "test_loss": [],
    }
#     loop throught train, and test step functions
    for epoch in tqdm(range(epochs)):
        
        train_acc, train_loss = train_step(model, train_dataloader, optimizer, loss_fn, device)
        
        test_acc, test_loss = test_step(model, test_dataloader, loss_fn, device)
        
#         print out the result
        print(
            f"Epoch: {epoch} | Train acc: {train_acc:.4f}, Train Loss: {train_loss:.4f} | Test acc: {test_acc:.4f}, Test Loss: {test_loss:.4f}"
        )
    
        result["train_acc"].append(train_acc)
        result["train_loss"].append(train_loss)
        result["test_acc"].append(test_acc)
        result["test_loss"].append(test_loss)
    return result

Overwriting modulers\engine.py


In [18]:
from pathlib import Path


def save_model(model: torch.nn.Module, target_dir: str, model_name:str):
    
    target_dir_path = Path(target_dir)
    target_dir_path.mkdir(parents=True, exist_ok=True)
    
    assert model_name.endswith("pth") or model_name.endswith("pt"), "model_name parameter should end with `pth` or `pt`."
    model_path = target_dir_path / model_name
    
    torch.save(model.state_dict(), model_path)
    
    print(f"The model was saved successfully in: {model_path}")

# 5.1. Creat a file called `utils.py` with utility functions

In [19]:
%%writefile "modulers\utils.py"
from pathlib import Path
import torch

def save_model(
    model: torch.nn.Module,
    target_dir: str,
    model_name:str
):
    """
    Saves the model's state dictionary to a specified directory.

    Args:
        model (torch.nn.Module): The neural network model to save.
        target_dir (str): The target directory where the model will be saved.
        model_name (str): The name of the model file. Must end with '.pth' or '.pt'.

    Raises:
        AssertionError: If `model_name` does not end with '.pth' or '.pt'.

    Returns:
        None

    Example usage:
        save_model(model, target_dir="models/", model_name="my_model.pth")
    """
    
    target_dir_path = Path(target_dir)
    target_dir_path.mkdir(parents=True, exist_ok=True)
    
    assert model_name.endswith("pth") or model_name.endswith("pt"), "model_name parameter should end with `pth` or `pt`."
    model_path = target_dir_path / model_name
    
    torch.save(model.state_dict(), model_path)
    
    print(f"The model was saved successfully in: {model_path}")



Overwriting modulers\utils.py


# 6. Train, Evaluate, and save the model

In [20]:
import torch

torch.manual_seed(42)
NUM_EPOCHS = 50

model_0 = TinyVGG(
    input_shape=3,
    hidden_units=10,
    output_shape=len(class_names)
).to(device)

optimizer = torch.optim.Adam(model_0.parameters(), lr=0.001)
loss_fn = torch.nn.CrossEntropyLoss()

# Start time
from timeit import default_timer as timer

start = timer()

# Train the model

model_0_result = train(
    model=model_0,
    train_dataloader=train_loader,
    test_dataloader=test_loader,
    loss_fn=loss_fn,
    optimizer=optimizer,
    epochs=NUM_EPOCHS,
    device=device
)

# End the timer
end = timer()

print(f"Total training time: {end - start:.4f} seconds")

# Save the model
save_model(
    model=model_0,
    target_dir="modulers/models/",
    model_name="going_moduler_cell_mode_tinyvgg_model.pth"
)

  0%|          | 0/50 [00:00<?, ?it/s]

Epoch: 0 | Train acc: 0.3047, Train Loss: 1.1063 | Test acc: 0.3116, Test Loss: 1.0983
Epoch: 1 | Train acc: 0.3320, Train Loss: 1.0995 | Test acc: 0.5417, Test Loss: 1.0698
Epoch: 2 | Train acc: 0.4922, Train Loss: 1.0862 | Test acc: 0.5227, Test Loss: 1.0799
Epoch: 3 | Train acc: 0.4102, Train Loss: 1.0826 | Test acc: 0.5729, Test Loss: 1.0598
Epoch: 4 | Train acc: 0.4141, Train Loss: 1.0631 | Test acc: 0.5540, Test Loss: 1.0612
Epoch: 5 | Train acc: 0.4570, Train Loss: 1.0214 | Test acc: 0.4820, Test Loss: 1.0069
Epoch: 6 | Train acc: 0.4258, Train Loss: 0.9661 | Test acc: 0.6042, Test Loss: 0.9268
Epoch: 7 | Train acc: 0.5938, Train Loss: 0.9246 | Test acc: 0.4953, Test Loss: 1.0317
Epoch: 8 | Train acc: 0.6172, Train Loss: 0.9082 | Test acc: 0.3532, Test Loss: 1.0718
Epoch: 9 | Train acc: 0.4609, Train Loss: 0.9925 | Test acc: 0.4233, Test Loss: 1.0331
Epoch: 10 | Train acc: 0.5312, Train Loss: 0.8468 | Test acc: 0.5540, Test Loss: 0.9897
Epoch: 11 | Train acc: 0.4375, Train Loss:

# 6.1. Train, Evaluate, and save the model script mode `train.py`

In [26]:
%%writefile "modulers\train.py"
import os
from timeit import default_timer as timer

import torch
from torchvision import transforms

import data_setup, model_builder, engine, utils

# Some Hyper-parameters
NUM_EPOCHS = 50
HIDDEN_UNITS = 10
BATCH_SIZE = 32
LEARNING_RATE = 0.001

# SET THE DEVICE
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Directories
train_dir = 'data/pizza_steak_sushi/train'
test_dir = 'data/pizza_steak_sushi/test'
TARGET_MODEDL_PATH = "modulers/model/"

# Create a tansform
data_transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor()
])

# Create our data loaders
train_loader, test_loader, class_names = data_setup.create_dataloader(
    train_dir=train_dir,
    test_dir=test_dir,
    transform=data_transform,
    batch_size=BATCH_SIZE
)

# Create a model
model = model_builder.TinyVGG(
    input_shape=3,
    hidden_units=HIDDEN_UNITS,
    output_shape=len(class_names)
).to(device)

# Setup loss and optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
loss_fn = torch.nn.CrossEntropyLoss()

# Start time
start = timer()

# Start training with engine file
result = engine.train(
    model=model,
    train_dataloader=train_loader,
    test_dataloader=test_loader,
    loss_fn=loss_fn,
    optimizer=optimizer,
    epochs=NUM_EPOCHS,
    device=device
)

# End the timer
end = timer()

print(f"Total training time: {end - start:.4f} seconds")

# Save the model
utils.save_model(
    model,
    TARGET_MODEDL_PATH,
    "going_moduler_script_mode_tinyvgg_model.pth"
)

Overwriting modulers\train.py


In [27]:
!python modulers/train.py

Epoch: 0 | Train acc: 0.2891, Train Loss: 1.1019 | Test acc: 0.2992, Test Loss: 1.0981
Epoch: 1 | Train acc: 0.2969, Train Loss: 1.0996 | Test acc: 0.1979, Test Loss: 1.1110
Epoch: 2 | Train acc: 0.2930, Train Loss: 1.0953 | Test acc: 0.1979, Test Loss: 1.1111
Epoch: 3 | Train acc: 0.2930, Train Loss: 1.0910 | Test acc: 0.2396, Test Loss: 1.1229
Epoch: 4 | Train acc: 0.4727, Train Loss: 1.1010 | Test acc: 0.2812, Test Loss: 1.1156
Epoch: 5 | Train acc: 0.4531, Train Loss: 1.0736 | Test acc: 0.4631, Test Loss: 1.0699
Epoch: 6 | Train acc: 0.4297, Train Loss: 1.0822 | Test acc: 0.6146, Test Loss: 1.0120
Epoch: 7 | Train acc: 0.5625, Train Loss: 1.0164 | Test acc: 0.3106, Test Loss: 1.0926
Epoch: 8 | Train acc: 0.5000, Train Loss: 1.0047 | Test acc: 0.3532, Test Loss: 1.0567
Epoch: 9 | Train acc: 0.6602, Train Loss: 0.9735 | Test acc: 0.2812, Test Loss: 1.0367
Epoch: 10 | Train acc: 0.6133, Train Loss: 0.9133 | Test acc: 0.3116, Test Loss: 1.0334
Epoch: 11 | Train acc: 0.6211, Train Loss:

2024-08-30 20:47:38.372740: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-30 20:47:39.397103: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-30 20:47:42.488248: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.

100%|##########| 50/50 [01:17<00:00,  1.55s/it]
